# **Open Source SW and Python Programming Project: Implementation of Subway Navigation**

In this project, you will implement a navigation system for Seoul subway stations, especially **step by step**




# 0. Initialization

Read subway station information by reading the file **simplified_subway_info_english.xlsx** or **simplified_subway_info_korean.xlsx**
*  The excel file contains subway station information of Seoul subway line 1 ~ 4
*  When you execute this sourcecode, you MUST upload these files in your Colab runtime environment
*   **MUST NOT** change this code cell

In [36]:
import xlrd

# Read data file
data = xlrd.open_workbook("simplified_subway_info_english.xlsx")
data = data.sheet_by_name('Sheet1')

# Store the loaded book object as a string list in subwayStation variable
subwayStation = []
for line in range(4) :
    cur = [x for x in data.col_values(line) if x]    
    subwayStation.append(cur[1:])

# 1. Your implementation
Let's start the implementation of a subway naviation system by using the loaded subway station information :)

*   You can access the subway station information by referring to **subwayStation** variable (list type)

In [ ]:
'''
This is an example code for showing how to use the loaded data.
You can remove this code.
'''
for i in range(4) :
    print(subwayStation[i]) # subwayStation[i] contatins the (i - 1)-th subway line information
